In [56]:
import pandas as pd
import numpy as np
import math
import matplotlib.pyplot as plt
import pickle
import datetime
import warnings
warnings.filterwarnings('ignore')

from xgboost import XGBClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from math import *
from sklearn.model_selection import train_test_split

x_centre = 3760901.5068
y_centre = -19238905.6133

train = pd.read_csv('data_train.csv')
test = pd.read_csv('data_test.csv')
train = train.drop(['Unnamed: 0'],axis=1)
test = test.drop(['Unnamed: 0'],axis=1)

def conv(time_1):   
    sec1 = 3600*int(time_1[0:2]) + 60*int(time_1[3:5]) + int(time_1[6:8])                                                                                                                                
    return (sec1)

def distance(x1,x2,y1,y2):
    dist = np.sqrt(((x2-x1)**2)+((y2-y1)**2))
    return dist

def ecartTemps(t1,t2):
    ecart= t2-t1
    return ecart

def distanceCentre(x,y):
    dx = max(0,abs(x_centre-x)-10000)
    dy = max(0,abs(y_centre-y)-30000)
    return (dx**2+dy**2)

ecartTemps = np.vectorize(ecartTemps)

distance = np.vectorize(distance)
distanceCentre = np.vectorize(distanceCentre)
#vitesses négatives



test.vmin[test.vmin < 0] = np.NaN
test.vmean[test.vmean < 0] = np.NaN
test.vmax[test.vmax < 0] = np.NaN

train.vmin[train.vmin < 0] = np.NaN
train.vmean[train.vmean < 0] = np.NaN
train.vmax[train.vmax < 0] = np.NaN

#incohérences
train['vmean'][(train.time_entry==train.time_exit)] = 0
train['vmin'][(train.time_entry==train.time_exit)] = 0
train['vmax'][(train.time_entry==train.time_exit)] = 0

test['vmean'][(test.time_entry==test.time_exit)] = 0
test['vmin'][(test.time_entry==test.time_exit)] = 0
test['vmax'][(test.time_entry==test.time_exit)] = 0

train['vmax'][(train.x_entry==train.x_exit)&(train.y_entry==train.y_exit)] = 0
train['vmin'][(train.x_entry==train.x_exit)&(train.y_entry==train.y_exit)] = 0
train['vmean'][(train.x_entry==train.x_exit)&(train.y_entry==train.y_exit)] = 0

test['vmax'][(test.x_entry==test.x_exit)&(test.y_entry==test.y_exit)] = 0
test['vmin'][(test.x_entry==test.x_exit)&(test.y_entry==test.y_exit)] = 0
test['vmean'][(test.x_entry==test.x_exit)&(test.y_entry==test.y_exit)] = 0

#incohérences 2 

# train['vmean'][(train.time_entry!=train.time_exit)&(train.vmean == 0)] = np.NaN
# train['vmin'][(train.time_entry!=train.time_exit)&(train.vmin == 0)] = np.NaN
# train['vmax'][(train.time_entry!=train.time_exit)&(train.vmax == 0)] = np.NaN

# test['vmean'][(test.time_entry!=test.time_exit)&(test.vmean == 0)] = np.NaN
# test['vmin'][(test.time_entry!=test.time_exit)&(test.vmin == 0)] = np.NaN
# test['vmax'][(test.time_entry!=test.time_exit)&(test.vmax == 0)] = np.NaN

#train['vmax'][((train.x_entry!=train.x_exit)|(train.y_entry!=train.y_exit))&(train.vmax == 0)] = np.NaN
#train['vmin'][((train.x_entry!=train.x_exit)|(train.y_entry!=train.y_exit))&(train.vmin == 0)] = np.NaN
#train['vmean'][((train.x_entry!=train.x_exit)|(train.y_entry!=train.y_exit))&(train.vmean == 0)] = np.NaN

#test['vmax'][((test.x_entry!=test.x_exit)|(test.y_entry!=test.y_exit))&(test.vmax == 0)] = np.NaN
#test['vmin'][((test.x_entry!=test.x_exit)|(test.y_entry!=test.y_exit))&(test.vmin == 0)] = np.NaN
#test['vmean'][((test.x_entry!=test.x_exit)|(test.y_entry!=test.y_exit))&(test.vmean == 0)] = np.NaN

#outliers
train.vmean = train.vmean.loc[(train.vmean<90)]
train.vmax = train.vmean.loc[(train.vmax<90)]
train.vmin = train.vmean.loc[(train.vmin<90)]


#nan by mean
# train['vmax']=train['vmax'].replace( np.NaN , train['vmax'].mean() )
# train['vmin']=train['vmin'].replace( np.NaN , train['vmin'].mean() )
# train['vmean']=train['vmean'].replace( np.NaN , train['vmean'].mean() )

# test['vmax']=test['vmax'].replace( np.NaN , test['vmax'].mean() )
# test['vmin']=test['vmin'].replace( np.NaN , test['vmin'].mean() )
# test['vmean']=test['vmean'].replace( np.NaN , test['vmean'].mean() )

# trainTempVmean = train.loc[(train.vmean>0)]
# trainTempVmean = trainTempVmean.dropna()
# trainTempVmean = trainTempVmean.vmean.mean()

# trainTempVmax = train.loc[(train.vmax>0)]
# trainTempVmax = trainTempVmax.dropna()
# trainTempVmax = trainTempVmax.vmax.mean()

# trainTempVmin = train.loc[(train.vmin>0)]
# trainTempVmin = trainTempVmin.dropna()
# trainTempVmin = trainTempVmin.vmin.mean()

# testTempVmean = test.loc[(test.vmean>0)]
# testTempVmean = testTempVmean.dropna()
# testTempVmean = testTempVmean.vmean.mean()

# testTempVmax = test.loc[(test.vmax>0)]
# testTempVmax = testTempVmax.dropna()
# testTempVmax = testTempVmax.vmax.mean()

# testTempVmin = test.loc[(test.vmin>0)]
# testTempVmin = testTempVmin.dropna()
# testTempVmin = testTempVmin.vmin.mean()


# train['vmax']=train['vmax'].replace( np.NaN , trainTempVmax)
# train['vmin']=train['vmin'].replace( np.NaN , trainTempVmin)
# train['vmean']=train['vmean'].replace( np.NaN , trainTempVmean)
# test['vmax']=test['vmax'].replace( np.NaN , testTempVmax)
# test['vmin']=test['vmin'].replace( np.NaN , testTempVmin)
# test['vmean']=test['vmean'].replace( np.NaN , testTempVmean)

vec_conv =  np.vectorize(conv)
train.time_entry=vec_conv(train.time_entry)
train.time_exit=vec_conv(train.time_exit)

test.time_entry=vec_conv(test.time_entry)
test.time_exit=vec_conv(test.time_exit)

#moments
# train['nuit']= 0
# train['nuit'][(train.time_entry<28800)]=1
# train['matin']= 0
# train['matin'][(train.time_entry<43200)&(train.time_entry>28800)]=1
# train['aprem']= 0
# train['aprem'][(train.time_entry>=43200)]=1

# test['nuit']= 0
# test['nuit'][(test.time_entry<28800)]=1
# test['matin']= 0
# test['matin'][(test.time_entry<43200)&(test.time_entry>28800)]=1
# test['aprem']= 0
# test['aprem'][(test.time_entry>=43200)]=1

train['duree']=ecartTemps(train.time_entry,train.time_exit)
test['duree']=ecartTemps(test.time_entry,test.time_exit)
train = train.drop(['time_entry','time_exit'],axis=1)
test = test.drop(['time_entry','time_exit'],axis=1)

#train['distCenter']=distanceCentre(train.x_entry,train.y_entry)
#test['distCenter']=distanceCentre(test.x_entry,test.y_entry)

train['distCenter_entry']=distanceCentre(train.x_entry,train.y_entry)
test['distCenter_entry']=distanceCentre(test.x_entry,test.y_entry)

train['distCenter_exit']=distanceCentre(train.x_exit,train.y_exit)
test['distCenter_exit']=distanceCentre(test.x_exit,test.y_exit)

train['Ecart_dist']=train['distCenter_exit']-train['distCenter_entry']
test['Ecart_dist']=test['distCenter_exit']-test['distCenter_entry']

train['distCenter_entry']=distanceCentre(train.x_entry,train.y_entry)
test['distCenter_entry']=distanceCentre(test.x_entry,test.y_entry)

train['distTrajet']=distance(train.x_entry,train.x_exit,train.y_entry,train.y_exit)
test['distTrajet']=distance(test.x_entry,test.x_exit,test.y_entry,test.y_exit)


#angles

# u = np.array([train.x_exit,train.y_exit])-np.array([train.x_entry,train.y_entry])
# v = np.array([x_centre-train.x_entry,y_centre-train.y_entry])

# angle_train = []
# for i in range(u.shape[1]):
#     scalar = (np.dot([u[0][i],u[1][i]],[v[0][i],v[1][i]]))
#     u_norm = (np.linalg.norm([u[0][i],u[1][i]]))
#     v_norm = (np.linalg.norm([v[0][i],v[1][i]]))
#     angle_train.append(degrees(acos(scalar/(u_norm*v_norm))))
    
# u = np.array([test.x_exit,test.y_exit])-np.array([test.x_entry,test.y_entry])
# v = np.array([x_centre-test.x_entry,y_centre-test.y_entry])

# angle_test = []
# for i in range(u.shape[1]):
#     scalar = (np.dot([u[0][i],u[1][i]],[v[0][i],v[1][i]]))
#     u_norm = (np.linalg.norm([u[0][i],u[1][i]]))
#     v_norm = (np.linalg.norm([v[0][i],v[1][i]]))
#     angle_test.append(degrees(acos(scalar/(u_norm*v_norm))))

#application

# train['angle']=angle_train
# test['angle']=angle_test
tr = test.loc[test.x_exit.isnull()]
train['city_center'] = 0
train['city_center'][(train.x_exit>=3750901.5068) & (train.x_exit<=3770901.5068)&(train.y_exit>=(-19268905.6133)) & (train.y_exit<=(-19208905.6133))]=1

# train.drop(['x_entry', 'x_exit','y_exit','y_entry'], axis=1, inplace=True)
# test.drop(['x_entry', 'x_exit','y_exit','y_entry'], axis=1, inplace=True)

In [57]:
#test=test.loc[test.x_exit.isnull()!=True]

TestVmeanNan = test.loc[(test.vmean.isnull()) & (test.x_exit.isnull()!=True)]
TestVmean = test.loc[test.vmean.isnull()!=True]
TestVminNan = test.loc[(test.vmin.isnull()) & (test.x_exit.isnull()!=True)]
TestVmin = test.loc[test.vmin.isnull()!=True]
TestVmaxNan = test.loc[(test.vmax.isnull()) & (test.x_exit.isnull()!=True)]
TestVmax = test.loc[test.vmax.isnull()!=True]

VmeanNan = train.loc[train.vmean.isnull()]
Vmean = train.loc[train.vmean.isnull()!=True]
VminNan = train.loc[train.vmin.isnull()]
Vmin = train.loc[train.vmin.isnull()!=True]
VmaxNan = train.loc[train.vmax.isnull()]
Vmax = train.loc[train.vmax.isnull()!=True]

In [58]:
TestVmeanNan.drop(['vmax', 'vmin','distCenter_exit','Ecart_dist','x_exit','y_exit','hash','trajectory_id'], axis=1, inplace=True)
TestVmean.drop(['vmax', 'vmin','distCenter_exit','Ecart_dist','x_exit','y_exit','hash','trajectory_id'], axis=1, inplace=True)

TestVminNan.drop(['vmax', 'vmean','distCenter_exit','Ecart_dist','x_exit','y_exit','hash','trajectory_id'], axis=1, inplace=True)
TestVmin.drop(['vmax', 'vmean','distCenter_exit','Ecart_dist','x_exit','y_exit','hash','trajectory_id'], axis=1, inplace=True)

TestVmaxNan.drop(['vmean', 'vmin','distCenter_exit','Ecart_dist','x_exit','y_exit','hash','trajectory_id'], axis=1, inplace=True)
TestVmax.drop(['vmean', 'vmin','distCenter_exit','Ecart_dist','x_exit','y_exit','hash','trajectory_id'], axis=1, inplace=True)

VmeanNan.drop(['vmax', 'vmin','distCenter_exit','Ecart_dist','city_center','x_exit','y_exit','hash','trajectory_id'], axis=1, inplace=True)
Vmean.drop(['vmax', 'vmin','distCenter_exit','Ecart_dist','city_center','x_exit','y_exit','hash','trajectory_id'], axis=1, inplace=True)

VminNan.drop(['vmax', 'vmean','distCenter_exit','Ecart_dist','city_center','x_exit','y_exit','hash','trajectory_id'], axis=1, inplace=True)
Vmin.drop(['vmax', 'vmean','distCenter_exit','Ecart_dist','city_center','x_exit','y_exit','hash','trajectory_id'], axis=1, inplace=True)

VmaxNan.drop(['vmean', 'vmin','distCenter_exit','Ecart_dist','city_center','x_exit','y_exit','hash','trajectory_id'], axis=1, inplace=True)
Vmax.drop(['vmean', 'vmin','distCenter_exit','Ecart_dist','city_center','x_exit','y_exit','hash','trajectory_id'], axis=1, inplace=True)

In [59]:
# NotNullVmean = Vmean.loc[Vmean.vmean!=0]
# NotNullVmax = Vmax.loc[Vmax.vmax!=0]
# NotNullVmin = Vmin.loc[Vmin.vmin!=0]

In [60]:
TestVmax = TestVmax.dropna()
TestVmin = TestVmin.dropna()
TestVmean = TestVmean.dropna()

VmaxConcat = pd.concat([TestVmax,Vmax])
VminConcat = pd.concat([TestVmin,Vmin])
VmeanConcat = pd.concat([TestVmean,Vmean])

In [61]:
from sklearn.linear_model import LinearRegression
from lightgbm import LGBMRegressor

Lgb = LGBMRegressor(n_estimators=300,max_depth=8)
Reg = LinearRegression(fit_intercept=True,normalize=True,n_jobs=-1)

# speeds = [Vmean,Vmin,Vmax]
speeds = [VmeanConcat,VminConcat,VmaxConcat]
#speeds = [NotNullVmean,NotNullVmin,NotNullVmax]
trainSpeedsNan =[VmeanNan,VminNan,VmaxNan]
testSpeedsNan =[TestVmeanNan,TestVminNan,TestVmaxNan]
TrainPreds=[]
TestPreds=[]
i=0

for speed in speeds:
    X = speed.iloc[:,1:].values
    Y = speed.iloc[:,0].values
    Lgb.fit(X,Y)
    TrainPreds.append(Lgb.predict(trainSpeedsNan[i].iloc[:,1:]))
    TestPreds.append(Lgb.predict(testSpeedsNan[i].iloc[:,1:]))
    i+=1

In [62]:
VmeanNan = train.loc[train.vmean.isnull()]
VminNan = train.loc[train.vmin.isnull()]
VmaxNan = train.loc[train.vmax.isnull()]
VmeanNan.vmean = TrainPreds[0]
VminNan.vmin = TrainPreds[1]
VmaxNan.vmax = TrainPreds[2]

In [63]:
TestVmeanNan = test.loc[(test.vmean.isnull()) & (test.x_exit.isnull()!=True)]
TestVminNan = test.loc[(test.vmin.isnull()) & (test.x_exit.isnull()!=True)]
TestVmaxNan = test.loc[(test.vmax.isnull()) & (test.x_exit.isnull()!=True)]

TestVmeanNan.vmean = TestPreds[0]
TestVminNan.vmin = TestPreds[1]
TestVmaxNan.vmax = TestPreds[2]

In [64]:
Vmean = pd.concat([Vmean.vmean,VmeanNan.vmean])
Vmean = pd.DataFrame(Vmean)
Vmean = Vmean.sort_index()
Vmean.to_csv("Vmean_comb_lgb.csv",index=False)

Vmin = pd.concat([Vmin.vmin,VminNan.vmin])
Vmin = pd.DataFrame(Vmin)
Vmin = Vmin.sort_index()
Vmin.to_csv("Vmin_comb_lgb.csv",index=False)

Vmax = pd.concat([Vmax.vmax,VmaxNan.vmax])
Vmax = pd.DataFrame(Vmax)
Vmax = Vmax.sort_index()
Vmax.to_csv("Vmax_comb_lgb.csv",index=False)

In [65]:
TestVmean = pd.concat([TestVmean.vmean,TestVmeanNan.vmean,tr.vmean])
TestVmean = pd.DataFrame(TestVmean)
TestVmean = TestVmean.sort_index()
TestVmean.to_csv("Test_Vmean_comb_lgb.csv",index=False)

TestVmin = pd.concat([TestVmin.vmin,TestVminNan.vmin,tr.vmin])
TestVmin = pd.DataFrame(TestVmin)
TestVmin = TestVmin.sort_index()
TestVmin.to_csv("Test_Vmin_comb_lgb.csv",index=False)

TestVmax = pd.concat([TestVmax.vmax,TestVmaxNan.vmax,tr.vmax])
TestVmax = pd.DataFrame(TestVmax)
TestVmax = TestVmax.sort_index()
TestVmax.to_csv("Test_Vmax_comb_lgb.csv",index=False)

In [87]:
def findSpeed(df):
    lesdataframes=[]

    for vincent in df.columns[1:]:
        print(vincent)
        k=df
        k = k.groupby('hash')[vincent].apply(list)

        if (vincent == 'x_exit') | (vincent == 'y_exit') :
            for i in range (k.shape[0]):
                k[i].reverse()
                k[i]=k[i][1:]      #x_exit y_exit  
        if (vincent == 'vmin') | (vincent == 'vmax') | (vincent == 'vmean'):
            for i in range (k.shape[0]):
                k[i].reverse()
                last = k[i][0]
                if isnan(last):
                    k[i][0]=-1
        else :
            for i in range (k.shape[0]):
                k[i].reverse()


        k=pd.DataFrame(k)


        # expand df.time_entry into its own dataframe
        gouz = k[vincent].apply(pd.Series)

        # rename each variable is time_entry
        gouz = gouz.rename(columns = lambda x : vincent + str(x))
        
        if (vincent == 'distCenter_entry'):
            gouz = gouz.fillna(math.inf) #arbre sinon il faudra mettre -1 ou tres grand
        else:
            gouz = gouz.fillna(0)
        
        
        lesdataframes.append(gouz)

    r = pd.concat(lesdataframes,axis=1)
    r = r.reset_index()
    r = r.drop(['hash'],axis=1)
    
    return r

In [72]:
train.vmax =Vmax
train.vmin =Vmin
train.vmean =Vmean

test.vmax =TestVmax
test.vmin =TestVmin
test.vmean =TestVmean
#supprimer et garder vmax_0 vmin et vmean pour tester

In [75]:
train.drop(['distCenter_exit','Ecart_dist','city_center','trajectory_id','distTrajet'], axis=1, inplace=True)
test.drop(['distCenter_exit','Ecart_dist','trajectory_id','distTrajet'], axis=1, inplace=True)

In [77]:
X = findSpeed(train)

vmax
vmin
vmean
x_entry
y_entry
x_exit
y_exit
duree
distCenter_entry
vmax
vmin
vmean
x_entry
y_entry
x_exit
y_exit
duree
distCenter_entry


In [88]:
X_test = findSpeed(test)

vmax
vmin
vmean
x_entry
y_entry
x_exit
y_exit
duree
distCenter_entry


In [90]:
Yvmax = X.vmax0
Yvmin = X.vmin0
Yvmean = X.vmean0

Yvmax.to_csv('TrainVmax0.csv',index=False)
Yvmin.to_csv('TrainVmin0.csv',index=False)
Yvmean.to_csv('TrainVmean0.csv',index=False)

X.drop(['vmax0', 'vmean0','vmin0'], axis=1, inplace=True)

In [131]:
ToKeepTestVmax = X_test.loc[X_test.vmax0!=-1].iloc[:,0]
ToPredTestVmax = X_test.loc[X_test.vmax0==-1].iloc[:,1:]
ToPredTestVmax.drop(ToPredTestVmax.columns[19:59],axis=1,inplace=True)

ToKeepTestVmin = X_test.loc[X_test.vmin0!=-1]['vmin0']
ToPredTestVmin = X_test.loc[X_test.vmin0==-1]
ToPredTestVmin.drop(ToPredTestVmin.columns[0:21],axis=1,inplace=True)
ToPredTestVmin.drop(ToPredTestVmin.columns[19:39],axis=1,inplace=True)

ToKeepTestVmean = X_test.loc[X_test.vmean0!=-1]['vmean0']
ToPredTestVmean = X_test.loc[X_test.vmean0==-1]
ToPredTestVmean.drop(ToPredTestVmean.columns[0:41],axis=1,inplace=True)

In [132]:
XVmax = X.drop(X.columns[19:57], axis=1)

XVmin = X.drop(X.columns[0:19], axis=1)
XVmin = XVmin.drop(XVmin.columns[19:38], axis=1)

XVmean = X.drop(X.columns[0:19], axis=1)
XVmean = XVmean.drop(XVmean.columns[0:19], axis=1)

In [133]:
from sklearn.model_selection import cross_val_score

lgb=LGBMRegressor(n_jobs=-1,n_estimators=500)
print('LGBM : ',(cross_val_score(lgb, XVmax, Yvmax, cv=10,verbose=3, scoring='r2',n_jobs=-1)).mean())
#0.5289348098049762 200 esti
#0.527016138207685 150 esti
#0.5310225033857917

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done   3 out of  10 | elapsed:  1.1min remaining:  2.5min
[Parallel(n_jobs=-1)]: Done   7 out of  10 | elapsed:  1.1min remaining:   27.5s


LGBM :  0.618111637116491


[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:  1.1min finished


In [194]:
# XVmax = XVmax.drop(XVmax.columns[-19:], axis=1)
# XVmax = XVmax.drop(XVmax.columns[98:-1], axis=1)
# XVmax = XVmax.drop(XVmax.columns[19:-2], axis=1)

In [218]:
import time
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedKFold

print(datetime.datetime.now())
X = XVmax
y = Yvmax
param_grid = {
    'n_jobs':[-1],
    'n_estimators': [300,500,1000],
    'learning_rate': [0.001,0.01,0.05,0.1],
    'max_depth': [5,6,8,10]
}

#kf = StratifiedKFold(n_splits=10,random_state=1,shuffle=True)
model = GridSearchCV(LGBMRegressor(),param_grid, cv=10,verbose=3, scoring= 'r2',n_jobs=-1)
model.fit(X,y)

print(model.best_score_)
print(model.best_params_)

print(datetime.datetime.now())

2019-05-04 21:09:16.234572
Fitting 10 folds for each of 48 candidates, totalling 480 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done   8 tasks      | elapsed:  1.2min
[Parallel(n_jobs=-1)]: Done 104 tasks      | elapsed: 20.9min
[Parallel(n_jobs=-1)]: Done 264 tasks      | elapsed: 47.3min
[Parallel(n_jobs=-1)]: Done 480 out of 480 | elapsed: 74.1min finished


0.535961409338884
{'learning_rate': 0.05, 'max_depth': 10, 'n_estimators': 1000, 'n_jobs': -1}
2019-05-04 22:23:37.397330
